# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

## 0. Initiate 

In [1]:
import pandas as pd
import numpy as np
import math
import json
from sklearn.preprocessing import MultiLabelBinarizer 
%matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

## 1. Clean and Organize Data

In [2]:
def clean_and_organize_data(portfolio=portfolio,profile=profile,transcript=transcript):
    ## Portfolio
    portfolio.rename(columns={'id':'offer_id'},inplace=True)
    mlb = MultiLabelBinarizer()
    portfolio = portfolio.join(pd.DataFrame(mlb.fit_transform(portfolio.pop('channels')),
                          columns=mlb.classes_,
                          index=portfolio.index))
    portfolio = portfolio.merge(pd.get_dummies(portfolio['offer_type'],prefix='offer',prefix_sep='::'),left_index=True,right_index=True)
    portfolio.drop(columns='offer_type',inplace=True)
    portfolio.set_index('offer_id')
    ## Profile
    profile = profile.dropna(axis = 0)
    profile.loc[:,'became_member_on']=pd.to_datetime(profile['became_member_on'],format='%Y%m%d')
    profile = profile.merge(pd.get_dummies(profile['gender'],prefix='gender',prefix_sep='::'),left_index=True,right_index=True)
    profile = profile.drop(columns='gender')
    profile.set_index('id',inplace=True)
    ## Transcript
    transcript = transcript.merge(transcript.value.apply(pd.Series),left_index=True,right_index=True)
    transcript.loc[transcript['offer_id'].isna(),'offer_id'] = transcript[transcript['offer_id'].isna()]['offer id']
    transcript = transcript.merge(pd.get_dummies(transcript['event'],prefix='event',prefix_sep='::'),left_index=True, right_index=True)
    transcript.drop(columns = ['offer id','value','event'],inplace=True)
    col_list = ['time','person',
                 'offer_id',
                 'amount',
                 'reward',
                 'event::offer completed',
                 'event::offer received',
                 'event::offer viewed',
                 'event::transaction']
    transcript = transcript[col_list]
    transcript = transcript[transcript['person'].isin(profile.index)].reset_index()
    
    return portfolio,profile,transcript

In [3]:
portfolio,profile,transcript = clean_and_organize_data()

C:\Users\thoma\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [4]:
portfolio.head() 

,difficulty,duration,offer_id,reward,email,mobile,social,web,offer::bogo,offer::discount,offer::informational
0,10,7,ae264e3637204a6fb9bb56bc8210ddfd,10,1,1,1,0,1,0,0
1,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,10,1,1,1,1,1,0,0
2,0,4,3f207df678b143eea3cee63160fa8bed,0,1,1,0,1,0,0,1
3,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,1,1,0,1,1,0,0
4,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,5,1,0,0,1,0,1,0


In [5]:
portfolio.shape

(10, 11)

In [6]:
profile.head() 

,age,became_member_on,income,gender::F,gender::M,gender::O
id,,,,,,
0610b486422d4921ae7d2bf64640c50b,55,2017-07-15,112000.0,1,0,0
78afa995795e4d85b5d9ceeca43f5fef,75,2017-05-09,100000.0,1,0,0
e2127556f4f64592b11af22de27a7932,68,2018-04-26,70000.0,0,1,0
389bc3fa690240e798340f5a15918d5c,65,2018-02-09,53000.0,0,1,0
2eeac8d8feae4a8cad5a6af0499a211d,58,2017-11-11,51000.0,0,1,0


In [7]:
profile.shape 

(14825, 6)

In [8]:
transcript.head()

,index,time,person,offer_id,amount,reward,event::offer completed,event::offer received,event::offer viewed,event::transaction
0,0,0,78afa995795e4d85b5d9ceeca43f5fef,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,0,1,0,0
1,2,0,e2127556f4f64592b11af22de27a7932,2906b810c7d4411798c6938adc9daaa5,NaN,NaN,0,1,0,0
2,5,0,389bc3fa690240e798340f5a15918d5c,f19421c1d4aa40978ebb69ca19b0e20d,NaN,NaN,0,1,0,0
3,7,0,2eeac8d8feae4a8cad5a6af0499a211d,3f207df678b143eea3cee63160fa8bed,NaN,NaN,0,1,0,0
4,8,0,aa4862eba776480b8bb9c68455b8c2e1,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,0,1,0,0


In [9]:
transcript.shape

(272762, 10)

## 2. Explore Data

In [21]:
test_id_1 ='78afa995795e4d85b5d9ceeca43f5fef'

In [22]:
transcript.columns.tolist()

['index',
 'time',
 'person',
 'offer_id',
 'amount',
 'reward',
 'event::offer received',
 'event::offer viewed',
 'event::transaction',
 'event::offer completed']

In [23]:
transcript = transcript[trans_cols]

In [25]:
transcript[transcript['person']==test_id_1]

,index,time,person,offer_id,amount,reward,event::offer received,event::offer viewed,event::transaction,event::offer completed
0,0,0,78afa995795e4d85b5d9ceeca43f5fef,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,1,0,0,0
13545,15561,6,78afa995795e4d85b5d9ceeca43f5fef,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,0,1,0,0
42328,47582,132,78afa995795e4d85b5d9ceeca43f5fef,NaN,19.89,NaN,0,0,1,0
42329,47583,132,78afa995795e4d85b5d9ceeca43f5fef,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,5.0,0,0,0,1
44065,49502,144,78afa995795e4d85b5d9ceeca43f5fef,NaN,17.78,NaN,0,0,1,0
47385,53176,168,78afa995795e4d85b5d9ceeca43f5fef,5a8bc65990b245e5a138643cd4eb9837,NaN,NaN,1,0,0,0
75802,85291,216,78afa995795e4d85b5d9ceeca43f5fef,5a8bc65990b245e5a138643cd4eb9837,NaN,NaN,0,1,0,0
77452,87134,222,78afa995795e4d85b5d9ceeca43f5fef,NaN,19.67,NaN,0,0,1,0
81938,92104,240,78afa995795e4d85b5d9ceeca43f5fef,NaN,29.72,NaN,0,0,1,0
125809,141566,378,78afa995795e4d85b5d9ceeca43f5fef,NaN,23.93,NaN,0,0,1,0
